# Load classes and test Oanda Account

In [1]:
# Import the necesary libraries
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import timedelta
import Oanda_Manager as om
import Signal_Gen as sg
import White_Strategy as ws
import Market_Simulator as MS
from Signal_Gen import Signal_Gen
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import timedelta
from pandas.tseries.offsets import BDay

In [2]:
# Create an Oanda_Manager object to connect to Oanda API
Account_details_file = 'Account_details.csv'
oandaAcc =  om.Oanda_Manager(Account_details_file)
print(oandaAcc.account_list())

(200, {'accounts': [{'id': '101-004-19105515-001', 'tags': []}]})


# Use Oanda_Manage.py to retrieve the historical data

In [3]:
# Retrieve the index data as a time-series dataframe usign an arbitrary date range 
#https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
asset_name="SPX500_USD"
granularity="H4"
#granularity="M"
from_date = pd.to_datetime('1-1-2003')
to_date =  pd.to_datetime(date.today())
candle_data = oandaAcc.get_all_candles_data(asset_name, granularity, from_date, to_date)
candle_data.tail()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
time,,,,,,,,,,,,,
2021-07-16 01:00:00,3478.0,4350.9,4362.5,4348.8,4361.8,4350.7,4362.3,4348.5,4361.6,4351.1,4362.7,4349.0,4362.0
2021-07-16 05:00:00,13190.0,4362.2,4367.0,4355.8,4366.6,4362.0,4366.8,4355.5,4366.4,4362.4,4367.3,4356.1,4366.8
2021-07-16 09:00:00,9175.0,4366.3,4375.0,4362.5,4373.9,4366.1,4374.8,4362.3,4373.7,4366.5,4375.3,4362.7,4374.1
2021-07-16 13:00:00,22743.0,4374.0,4378.0,4344.6,4352.4,4373.7,4377.8,4344.3,4352.2,4374.3,4378.3,4344.8,4352.5
2021-07-16 17:00:00,15200.0,4352.6,4356.0,4324.5,4328.3,4352.5,4355.9,4324.3,4328.1,4352.8,4356.2,4324.7,4328.5


In [4]:
# Describe the data
index_stats = candle_data.describe()
index_stats = index_stats.transpose()
index_stats

,count,mean,std,min,25%,50%,75%,max
volume,28034.0,3629.661697,8071.208796,1.0,363.000,1289.00,4118.500,203425.0
mid_o,28034.0,1811.896340,787.263820,674.7,1207.325,1464.30,2263.400,4391.4
mid_h,28034.0,1816.423532,788.887753,678.4,1210.700,1467.80,2266.700,4394.6
mid_l,28034.0,1807.170971,785.488241,666.3,1204.700,1460.70,2261.150,4384.3
mid_c,28034.0,1812.052433,787.413726,675.0,1207.300,1464.30,2263.900,4391.2
bid_o,28034.0,1811.736029,787.165447,674.7,1207.325,1464.25,2263.100,4391.2
bid_h,28034.0,1816.280666,788.805549,678.4,1210.700,1467.75,2266.400,4394.5
bid_l,28034.0,1807.012647,785.394183,666.3,1204.700,1460.45,2260.800,4384.1
bid_c,28034.0,1811.907892,787.331252,675.0,1207.300,1464.20,2263.625,4391.1
ask_o,28034.0,1812.056394,787.361826,674.7,1207.325,1464.40,2263.600,4391.7


In [5]:
#Appending time, mid_c and volume to a new dataframe
SPX500_USD_H4 = candle_data['mid_c']
SPX500_USD_H4 = SPX500_USD_H4.to_frame().merge(candle_data['volume'], on=candle_data.index, how='left')
SPX500_USD_H4 = SPX500_USD_H4.rename(columns={'key_0': 'time'})
SPX500_USD_H4 = SPX500_USD_H4.set_index('time')
SPX500_USD_H4

,mid_c,volume
time,,
2003-03-21 18:00:00,895.3,195.0
2003-03-24 06:00:00,879.4,182.0
2003-03-24 10:00:00,877.9,208.0
2003-03-24 14:00:00,867.9,238.0
2003-03-24 18:00:00,864.4,195.0
...,...,...
2021-07-16 01:00:00,4361.8,3478.0
2021-07-16 05:00:00,4366.6,13190.0
2021-07-16 09:00:00,4373.9,9175.0


In [8]:
# Retrieve and Plot the medium closing price
sp500_data = go.Scatter(x=SPX500_USD_H4.index, y=SPX500_USD_H4.mid_c)
layout = go.Layout(title='Price Plot', xaxis=dict(title='Date'), yaxis=dict(title='price'))
fig = go.Figure(data=[sp500_data], layout=layout)
fig.show()

In [9]:
#Store the cleanned data in the requested file
fileName= "SPX500_USD_H4.CSV"
oandaAcc.save_CSV_file(SPX500_USD_H4,fileName)

# Generate the signals and store them in a CSV file

In [ ]:
#Create a Signal_Gen object with the stored file
fileName= "SPX500_USD_H4.CSV"
Signal_Generator = Signal_Gen(fileName)
SPX500_H4_signals=Signal_Generator.series
SPX500_H4_signals.head()

In [ ]:
# Calculate the moving averages 14 21 40
rolling_window=14
SPX500_H4_signals = Signal_Generator.Simple_moving_average(rolling_window, 'SMA_14')
rolling_window=21
SPX500_H4_signals = Signal_Generator.Simple_moving_average(rolling_window, 'SMA_21')
rolling_window=40
SPX500_H4_signals = Signal_Generator.Simple_moving_average(rolling_window, 'SMA_40')
# Calculate the slope with three months window 6*90 =540
rolling_window=540
SPX500_H4_signals = Signal_Generator.moving_slope(rolling_window, 'Slope_540')

In [ ]:
# Plot the index and the generated signals

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.mid_c,
                    mode='lines', name='Close Price', line=dict(color='dimgrey', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.SMA_14,
                    mode='lines', name='SMA_14', line=dict(color='crimson', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.SMA_21,
                    mode='lines', name='SMA_21', line=dict(color='darkgreen', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.SMA_40,
                    mode='lines', name='SMA_40', line=dict(color='cornflowerblue', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=SPX500_H4_signals.index, y=SPX500_H4_signals.Slope_540,
                    mode='lines',name='Slope_540',line=dict(color='darkorange', width=2)), row=2, col=1)

fig.update_layout(autosize=False, width=1100,height=600)

fig.show()

In [ ]:
#Store the signals in the requested file
fileName= "SPX500_H4_SIGNALS.CSV"
Signal_Generator.save_CSV_file(SPX500_H4_signals,fileName)

# Execute the white robot strategy

In [ ]:
fileName= "SPX500_H4_SIGNALS.CSV"
White_Strat  = ws.White_Strategy(fileName)
#White_Strat.series.tail()

In [ ]:
oder_book = White_Strat.simple_white_strat()

In [ ]:
#Store the signals in the requested file
fileName= "WR_ORDER_BOOK.CSV"
White_Strat.save_CSV_file(oder_book,fileName)
oder_book.tail()

# Run Market Simulator - White Robot

In [ ]:
order_book = pd.read_csv(fileName)
Cash = 10000
start_date = '09/04/1995'
end_date = '09/05/1995'
MS = MS.Market_Sim(order_book, Cash, start_date, end_date)

In [ ]:
Portfolio_Simulation = MS.csv_outputs()

## Visualising the Portfolio Simulation

In [ ]:
Portfolio_Simulation = pd.read_csv('Portfolio_Simulation.csv')
Portfolio_Simulation.head()

In [ ]:
new_fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

In [ ]:
new_fig.add_trace(go.Scatter(x=Portfolio_Simulation.time, y=Portfolio_Simulation.mid_c,
                    mode='lines', name='Close Price', line=dict(color='dimgrey', width=2)), row=1, col=1)
new_fig.add_trace(go.Scatter(x=Portfolio_Simulation.time, y =Portfolio_Simulation.Signals,
                     mode='lines', name='Signals', line=dict(color='blue', width=2)), row=2, col=1)
new_fig.add_trace(go.Scatter(x=Portfolio_Simulation.time, y =Portfolio_Simulation['Portfolio Value'],
                     mode='lines', name='Portfolio Value', line=dict(color='red', width=2)), row=3, col=1)
new_fig.update_layout(autosize=False, width=1100,height=600)